In [2]:
threshold = 0.05

In [3]:
def Analysis(values, thr=0.05):
    # print(values)
    better = []
    comparable = []
    thr = 0.05 

    last_value = values[4]
    
    for v in values[0:4]:
        # print(v)
        better.append(round(last_value -v, 2) > thr)
        comparable.append(abs(round(last_value -v, 2)) <= thr)

    if all(better):
        return (True, 'better_all' )
    elif True in better:
        return (True, 'better_one' )
    elif all( comparable):
        return (True, 'Comp_with_all' )
    elif True in comparable:
        return (True, 'Comp_with_one' )

In [7]:
Analysis([0.84, 0.84, 0.85, 0.95, 0.89], thr=0.05)

(True, 'Comp_with_one')

<!-- Blues
BuGn
BuPu
GnBu
Greens
Greys
Oranges
OrRd
PuBu
PuBuGn
PuRd
Purples
RdPu
Reds
YlGn
YlGnBu
YlOrBr
YlOrRd -->


In [76]:
import pandas as pd
import plotly.graph_objects as go
import os
import glob
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

color_scale=[
    [0, 'green'],    # Value -1 will be red
    [0.5, 'red'], # Value 0 will be yellow
    [1, 'yellow']    # Value 1 will be blue
]

# Define the color scale constant
COLOR_SCALE = {
    'Comp_with_all': 'blue',
    'better_all': 'violet',
    'Comp_with_one': 'black',
    'better_one': 'red'
}
clm_list = [1, 6, 11, 16, 21]

def ResultSummary(file):
    # print(file)
    new_DF = pd.read_csv(file, sep='\t')
    new_DF.set_index('name', inplace=True)

    DF = new_DF.T
    DF.columns = new_DF.index
    DF.index = new_DF.columns
    df = DF.iloc[clm_list]

    column_anno_per = {}
    comparable = {}

    # print(df.columns)

    for n in df.columns.to_list():
        # print(n, df[n].values)
        comparable[n] = Analysis(df[n].values, threshold)
    return comparable

def Plot(number, width=300, height=400, saveSVG=False, file_type=None, color_labels='Greens'):
    input_file = fs[number]
    # print(input_file)
    threshold = 0.05

    result_1 = ResultSummary(input_file)

    true_columns = []
    true_column_comp = []

    for i, k in enumerate(result_1.keys()):
        if result_1[k]:
            true_column_comp.append((i, result_1[k], k))

    plotting_columns = {
        'Comp_with_all': [],
        'better_all': [],
        'Comp_with_one': [],
        'better_one': [],
        'None': [],
    }

    colors = COLOR_SCALE
    arranged_columns = []
    counter = 0

    for c in colors.keys():
        for i, a in enumerate(true_column_comp):
            if c == a[1][1]:
                counter += 1
                plotting_columns[c].append((a[2], counter - 1))
                arranged_columns.append(a[2])

    # DF = pd.read_csv(input_file, sep='\t')
    # DF.set_index('name', inplace=True)

    new_DF = pd.read_csv(input_file, sep='\t')
    new_DF.set_index('name', inplace=True)


    # Assuming you have your heatmap data


    DF = new_DF.T
    DF.columns = new_DF.index
    DF.index = new_DF.columns
    df = DF.iloc[clm_list]  # Acc
    df = df[arranged_columns]

    df.index.name = 'name'



    print(height, width)

    heatmap = go.Heatmap(
        z=df.values,
        x=df.columns,
        zmin=0,
        zmax=1,
        y=['LRC', 'DTC', 'SVC', 'RFC', 'HDC'],
        # colorbar=dict(title='Value'),
        text=df.values,  # Display values in each cell
        texttemplate="%{text}",  # Format for text
        textfont=dict(color='white'),  # Ensure text is visible on darker colors
        # colorscale='Greens'  
        colorscale=color_labels 
    )

    shapes = []

    for i in range(5, len(df), 5):
        shapes.append(
            go.layout.Shape(
                type='line',
                x0=-0.5,
                x1=len(df.columns) - 0.5,
                y0=i - 0.5,
                y1=i - 0.5,
                line=dict(color='white', width=1),
            )
        )

    ind = 0
    for t in plotting_columns.keys():
        if t != 'None' and len(plotting_columns[t]) > 0:
            col_idx = plotting_columns[t][0][1]
            row_idx = 4
            shape1 = go.layout.Shape(
                type='rect',
                x0=col_idx - 0.48,
                x1=plotting_columns[t][-1][1] + 0.48,
                y0=row_idx - 4.5,
                y1=row_idx + 0.5,
                line=dict(color=colors[t], width=2.5),  # Use color from the color scale constant
                fillcolor='rgba(255, 255, 255, 0)',  # Transparent fill
            )
            shapes.append(shape1)

    fig = go.Figure(data=[heatmap])

    # rows = df.shape[0]
    # cols = df.shape[1]
    
    # # Calculate the desired aspect ratio
    # cell_size = 200  # You can adjust this value for larger or smaller squares
    # height = rows * cell_size
    # width = cols * cell_size
    # print(height, width)

    fig.update_layout(
        width=width,
        height=height,
        shapes=shapes,
        title=input_file.split('/')[5].split('.')[0],
        xaxis=dict(title='Study'),
        yaxis=dict(title='Classifier'),
        yaxis_autorange='reversed',
        # colorscale=[[1, 'blue'], [-1, 'red']],
        autosize=False,
    )
    out_svg_file_path = '../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG' 


    if saveSVG:
        print(os.path.join(out_svg_file_path, input_file.split('/')[5].split('.')[0]+'_F1.'+file_type))
        fig.write_image(os.path.join(out_svg_file_path, input_file.split('/')[5].split('.')[0]+'_F1.'+file_type),  scale=2, format=file_type)
        fig.write_image(os.path.join(out_svg_file_path, input_file.split('/')[5].split('.')[0]+'_F1.'+'png'),  scale=2, format='png')
    
    fig.show()

In [38]:

fs = glob.glob('../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/*.tsv')

category_list = []
for i, f in enumerate(fs):
    category_list.append(f.split('/')[5].strip('.tsv'))
    print(i, f.split('/')[5].strip('.tsv'))

# Error fix


# Data Stats
# Color blind

0 born_method
1 history_of_periodontiti
2 disease_subtype
3 udy_condition
4 ORR
5 ever_smoker
6 PFS12
7 birth_order
8 country
9 feeding_practice
10 non_westernized
11 age_category
12 family_role
13 location
14 uncurated_metadata
15 gender
16 body_site
17 birth_control_pil
18 moker
19 dental_sample_type


In [39]:
pwd

'/Users/joshij/Desktop/Classification_5_06_24/Splitted/Fixed'

In [78]:


# ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',  'blackbody', 'bluered', 'blues', 'blugrn',
# 'bluyl', 'brbg', 'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',    'darkmint', 'deep', 'delta',
# 'dense', 'earth', 'edge', 'electric',  'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',   'haline',
#  'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',   'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges', 
#  'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu',
#  'pubugn',       'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 
# 'redor', 'reds', 'solar','spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose', 'tempo', 'temps',
#  'thermal', 'tropic', 'turbid', 'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd']"

Plot(19,  100, 400, saveSVG=True, file_type='svg', color_labels='emrld')

#curl, inferno, emrld, blues, blugrn, sunsetdark, plasma, oryel, speed, tealgrn,  deep, phase, aggrnyl, agsunset, algae, amp, bluyl, cividis, darkmint, deep, delta, gnbu, greens, haline, hsv, matter, oranges, orrd, portland, rdpu
#reds  solar speed sunset sunsetdark  thermal twilight  viridis ylgnbu ylorbr ylorrd

200 100
../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG/dental_sample_type_F1.svg


In [21]:
!ls ../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG 
# fig.write_image(input_file.split('.')[0]+'_ACC.svg', format="svg")
# input_file

ORR_ACC.svg                      ever_smoker_ACC.svg
PFS12_ACC.svg                    family_role_ACC.svg
age_category_ACC.svg             feeding_practice_ACC.svg
birth_control_pil_ACC.svg        gender_ACC.svg
birth_order_ACC.svg              history_of_periodontitis_ACC.svg
body_site_ACC.svg                location_ACC.svg
born_method_ACC.svg              non_westernized_ACC.svg
country_ACC.svg                  smoker_ACC.svg
dental_sample_type_ACC.svg       study_condition_ACC.svg
disease_subtype_ACC.svg          uncurated_metadata_ACC.svg


In [22]:

def Analysis(values, threshold=0.05):
    better = []
    comparable = []
    thr = 0.05 

    last_value = values[4]

    for v in values[0:4]:
        better.append(round(last_value -v, 2) > threshold)
        comparable.append(abs(round(last_value -v, 2)) <= threshold)

    if all(better):
        return (True, 'better_all' )
    elif True in better:
        return (True, 'better_one' )
    elif all( comparable):
        return (True, 'Comp_with_all' )
    elif True in comparable:
        return (True, 'Comp_with_one' )

In [23]:
# !ls ../../MicroBiomML/datasets-ra/

data_file = glob.glob('../../MicroBiomML/datasets-ra/*.tsv')

cut_off = 200

refined_catogories = []

f = open('../../ResultsSelectedMin200Samples/selected_data_'+str(cut_off)+'_Samples_or_more.txt', 'w')

for d in data_file:
    df = pd.read_csv(d, sep="\t")
    label_counts = df['label'].value_counts()
    count_values = label_counts.values
    if df.shape[0] >= cut_off:
    # print(d.split('/')[4].split('.')[1],df.shape[0], count_values[0], df.shape[0]/count_values[0])
    # print(d.split('/')[4].split('.')[1],df.shape[0], count_values[1], df.shape[0]/count_values[1])
        # print(d.split('/')[4], np.array([df.shape[0]/count_values[0], df.shape[0]/count_values[1]]).max(), df.shape[0], count_values[0], count_values[1] )
        refined_catogories.append(d.split('/')[4].split('.')[3])
        f.write(d.split('/')[4]+'\n')
    # print(d.split('/')[4].split('.')[1],df.shape[0], count_values[0], count_values[1], (count_values[0]+count_values[1])/2, (df.shape[0])/(count_values[0])+(df.shape[0])/(count_values[1])/2 )

f.close()
set(refined_catogories)

set()

In [24]:
cut_off  = 200
f = open('../../ResultsSelectedMin200Samples/selected_data_'+str(cut_off)+'_Samples_or_more.txt')
selected_lines = f.readlines()

In [25]:
out_file = open('../../ResultsSelectedMin200Samples/merged_results_'+str(cut_off)+'.tsv', 'w')

header = ['Acc_LRC','F1_LRC','Prec_LRC','Recall_LRC','MCC_LRC', 'Acc_DTC','F1_DTC','Prec_DTC','Recall_DTC','MCC_DTC', 'Acc_SVC','F1_SVC','Prec_SVC','Recall_SVC','MCC_SVC', 'Acc_RFC','F1_RFC','Prec_RFC','Recall_RFC','MCC_RFC','Acc_HDC','F1_HDC','Prec_HDC','Recall_HDC','MCC_HDC']
header = '\t'.join(header)

out_file.write('name'+'\t'+header+'\n')

def return_hdc_data(file_name):
    f = open('../../MicroBiomML/hdlib-ra-levels1000-retrain20-cv5-feature-selection/HDC_result_files/'+file_name+'.txt')
    lines = f.readlines()
    for i, line in enumerate(lines):
        if 'Total elapsed time:' in line:
            result = [f"{float(lines[i-5].split(' ')[1]):.2f}",  f"{float(lines[i-4].split(' ')[1]):.2f}", f"{float(lines[i-3].split(' ')[1]):.2f}", f"{float(lines[i-2].split(' ')[1]):.2f}", f"{float(lines[i-1].split(' ')[3]):.2f}"]
    return result

fs = glob.glob('./datasets-ra/*.tsv')

algs = ['LRC', 'DTC', 'SVC', 'RFC']

for d in selected_lines:
    d = d.strip('\n')

    result_list = []
    print("#######################")
    for alg in algs:
        data =  open(os.path.join('../../', alg+"_results", 'result.tsv'))
        lines = data.readlines()
        print(alg)
        for i in lines:
            if d in i:
                result_list = result_list + i.rstrip('\n').split(',')[1:]
    result_list =  [f"{float(v):.2f}" for v in result_list]
    
    result = return_hdc_data(d)
    out_file.write(d+'\t'+'\t'.join(result_list+result)+'\n')
out_file.close()


In [26]:
!pwd
!ls ../

/Users/joshij/Desktop/Classification_5_06_24/Splitted/Fixed
Analysis_with_mean.ipynb     dental_sample_type.tsv
Analysis_without_mean.ipynb  disease_subtype.tsv
Fixed                        ever_smoker.tsv
MeanEx.py                    family_role.tsv
ORR.tsv                      feeding_practice.tsv
PFS12.tsv                    gender.tsv
Untitled.ipynb               history_of_periodontitis.tsv
Untitled1.ipynb              location.tsv
Untitled2.ipynb              non_westernized.tsv
age_category.tsv             plot.py
alcohol.tsv                  remission.tsv
anti_PD_1.tsv                scatter_plot.svg
birth_control_pil.tsv        shigatoxin_2_elisa.tsv
birth_order.tsv              smoker.tsv
body_site.tsv                split_results.py
body_subsite.tsv             study_condition.tsv
born_method.tsv              treatment.tsv
country.tsv                  uncurated_metadata.tsv


In [28]:
f = open('../../ResultsSelectedMin200Samples/CombinedResults200/merged_results_200.tsv')

lines = f.readlines()
category =  list(set([ line.split('\t')[0].split('.')[3] for line in lines[1:] ]))

'LRC', 'DTC', 'LRC', 'RFC'

header = ['Acc_LRC','F1_LRC','Prec_LRC','Recall_LRC','MCC_LRC', 'Acc_DTC','F1_DTC','Prec_DTC','Recall_DTC','MCC_DTC', 'Acc_SVC','F1_SVC','Prec_SVC','Recall_SVC','MCC_SVC', 'Acc_RFC','F1_RFC','Prec_RFC','Recall_RFC','MCC_RFC','Acc_HDC','F1_HDC','Prec_HDC','Recall_HDC','MCC_HDC']
header = '\t'.join(header)


for c in category:
    out = open('../../ResultsSelectedMin200Samples/CombinedResults200/Splitted/'+c+'.tsv', 'w')
    out.write('name'+'\t'+header+'\n')

    for l in lines:
        if c in l:
            rename_list = [l.split('\t')[0].split('.')[1]] + l.split('\t')[1:]
            # print("\t".join(rename_list))
            out.write("\t".join(rename_list))
            # print(l)
out.close()
f.close()
   

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# # Sample data
# import pandas as pd

# # Sample data
# data = {
#     'Category': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
#     'Positive': [150, 300, 200, 250, 150, 100, 50, 30, 20],
#     'Negative': [200, 400, 300, 350, 250, 150, 100, 60, 40]
# }

# import matplotlib.pyplot as plt
# import numpy as np

# # Set the positions and width for the bars
# positions = np.arange(len(df['Category']))
# width = 0.4

# # Plot the data
# fig, ax = plt.subplots(figsize=(12, 8))

# # Plot Positive data on the left
# ax.barh(positions - width/2, df['Positive'], width, label='Positive', color='green')

# # Plot Negative data on the right
# ax.barh(positions + width/2, -df['Negative'], width, label='Negative', color='red')

# # Customize the plot
# ax.set(yticks=positions, yticklabels=df['Category'], xlabel='Sample Size', ylabel='Category')
# ax.axvline(0, color='black', linewidth=0.8)
# ax.legend()

# # Set custom x-ticks
# ticks = np.arange(-max(df['Negative']), max(df['Positive']) + 1, 50)
# ax.set_xticks(ticks)
# ax.set_xticklabels([abs(int(tick)) for tick in ticks])

# # Add labels to bars
# for index, value in enumerate(df['Positive']):
#     ax.text(value + 10, index - width/2, str(value), va='center', ha='left', color='green')

# for index, value in enumerate(df['Negative']):
#     ax.text(-value - 10, index + width/2, str(value), va='center', ha='right', color='red')

# plt.title('Sample Size Distribution by Category and Class')
# plt.show()



In [29]:
# Define the values
values = np.array([1.2, 2.3, 3.5, 4.5, 3.2, 2.1, 4.0, 1.5])

# Calculate the min and max values
min_value = values.min()
max_value = values.max()

# Apply min-max normalization
normalized_values = (values - min_value) / (max_value - min_value)

# Print the normalized values
normalized_values

array([0.        , 0.33333333, 0.6969697 , 1.        , 0.60606061,
       0.27272727, 0.84848485, 0.09090909])

In [30]:
# Define the unbalance scores
unbalance_scores = np.array([1.8, 2, 2.5, 3, 2.2, 4, 5, 2.8, 1.5])

def normalize_unbalance_scores(scores):
    min_score = 2
    max_score = scores.max()
    normalized_scores = abs((scores - min_score) / (max_score - min_score))
    return normalized_scores

# Apply the normalization
normalized_unbalance_scores = normalize_unbalance_scores(unbalance_scores)

# Print the normalized unbalance scores
normalized_unbalance_scores

array([0.06666667, 0.        , 0.16666667, 0.33333333, 0.06666667,
       0.66666667, 1.        , 0.26666667, 0.16666667])

In [31]:
dataset = open('../../ResultsSelectedMin200Samples/CombinedResults100/merged_results_100.tsv')
lines = dataset.readlines()
catagories = list(set([l.split('\t')[0].split('.')[3] for l in lines[1:]]))

cat_dict = {}
class_labels = []

for d in catagories:
    data_list = []
    for line in lines[1:]:
        if d in line:
            df = pd.read_csv('../../MicroBiomML_NonGit/datasets-ra/'+line.split('\t')[0], sep="\t")
            label_counts = df['label'].value_counts().to_list()
            # print(line.split('\t')[0])
            data_list.append({line.split('\t')[0]:label_counts, 'class_labels':list(set(df['label'].to_list()))})

    cat_dict[d] =  data_list

In [146]:
unique_labels = []
positives = []
negatives = []

cat_index = 20

for cat in [catagories[cat_index]]:
    for c in cat_dict[cat]:
        unique_labels.append(list(c.keys())[0].split('.')[3]+"_"+list(c.keys())[0].split('.')[0]+"_"+list(c.keys())[0].split('.')[1])
        positives.append(c[list(c.keys())[0]][0])
        negatives.append(c[list(c.keys())[0]][1])

# print(unique_labels)
#     print(positives)
#     print(negatives)
# print(catagories)
# Sample data

data = {
    'Category': unique_labels,
    'Class 1': positives,
    'Class 2': negatives
}

df = pd.DataFrame(data)

# Set the positions and width for the bars
positions = np.arange(len(df['Category']))
width = 0.4

# Plot the data
fig, ax = plt.subplots(figsize=(22, 2))
fig.subplots_adjust(left=0.35, right=0.95, top=0.8, bottom=0.3)

# Plot Positive data on the left
ax.barh(positions - width/2, df['Class 1'], width, label='Class 1', color='green')

# Plot Negative data on the right
ax.barh(positions + width/2, -df['Class 2'], width, label='Class 2', color='red')

# Customize the plot
ax.set(yticks=positions, yticklabels=df['Category'], xlabel='Sample Size')
# ax.set(yticks=positions, yticklabels=df['Category'], xlabel='Sample Size', ylabel='Category')

ax.axvline(0, color='black', linewidth=0.8)
ax.legend()

# Set custom x-ticks
ticks = np.arange(-max(df['Class 2']), max(df['Class 1']) + 1, 50)
ax.set_xticks(ticks)
ax.set_xticklabels([abs(int(tick)) for tick in ticks])

plt.tick_params(axis='both', which='major', labelsize=16)
plt.xlabel('Sample Size', fontsize=14)

# Add labels to bars
for index, value in enumerate(df['Class 1']):
    ax.text(value + 0.5, index - width/2, str(value), va='center', ha='left', color='green', fontsize=14)

for index, value in enumerate(df['Class 2']):
    ax.text(-value - 0.5, index + width/2, str(value), va='center', ha='right', color='red', fontsize=14)

plt.title('Sample Size Distribution: '+catagories[cat_index])
plt.savefig(os.path.join('../../DataDistributionPlot/',catagories[cat_index]+'.png'), dpi=600)
plt.show()

IndexError: list index out of range

In [8]:
!ls ../../DataDistributionPlot/

In [78]:

fs = glob.glob('/Users/joshij/Desktop/Classification_5_06_24/datasets-ra/*.tsv')

for c in category_list:
    counter = 0
    class_labels = None
    print("##############################")
    print(c)
    for f in fs: 
        if c in f:
            NewDf = pd.read_csv(f, sep='\t')
            if NewDf.shape[0] >= 100:
                counter = counter + NewDf.shape[0]
                class_labels = NewDf['label'].value_counts()
                # print(c , f)
                print(f.split('/')[6].split('.')[1])
                print(class_labels)

    print("Total Samples", counter)

##############################
born_method
ShaoY_2019
c_section    695
vaginal      638
Name: label, dtype: int64
BrooksB_2017
c_section    278
vaginal      125
Name: label, dtype: int64
BrooksB_2017
c_section    278
vaginal      125
Name: label, dtype: int64
Total Samples 2139
##############################
history_of_periodontiti
GhensiP_2019
no     62
yes    43
Name: label, dtype: int64
Total Samples 105
##############################
disease_subtype
LiJ_2014
CD    76
UC    64
Name: label, dtype: int64
HMP_2019_ibdmdb
CD    625
UC    426
Name: label, dtype: int64
HallAB_2017
UC    97
CD    88
Name: label, dtype: int64
HallAB_2017
UC    97
CD    88
Name: label, dtype: int64
HMP_2019_ibdmdb
CD    625
UC    426
Name: label, dtype: int64
Total Samples 2612
##############################
udy_condition
NielsenHB_2014
control    248
IBD        148
Name: label, dtype: int64
ZhuF_2020
schizophrenia    90
control          81
Name: label, dtype: int64
VogtmannE_2016
control    52
CRC        52

In [57]:
born_method
history_of_periodontiti
disease_subtype
study_condition
ever_smoker
birth_order
country
feeding_practice
non_westernized
age_category
family_role
location
uncurated_metadata
gender
body_site
birth_control_pil
smoker
dental_sample_type

NameError: name 'born_method' is not defined

In [145]:
#box plot for data summary
#read DataSummary.txt file
data_summary = open('../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG/DataSummary.txt')
lines = data_summary.readlines()

dataBalance = []

for i, line in enumerate(lines):

    values = []
    if 'Name:' in line:
        values.append(int(lines[i-1].strip('\n').split(' ')[-1]))
        values.append(int(lines[i-2].strip('\n').split(' ')[-1]))
        # print(lines[i-1].strip('\n').split(' ')[0], lines[i-1].strip('\n').split(' ')[-1])
        # print(lines[i-1].strip('\n').split(' ')[0], lines[i-2].strip('\n').split(' ')[-1])
        # print(values)
        #Extract min and max values from values
        min_value = min(values)
        max_value = max(values)
        #Apply min-max normalization
        normalized_values = round((min_value / sum(values)), 3)
        dataBalance.append(normalized_values )
        # print(normalized_values)
print(dataBalance)
#plot box plot from dataBalance
normalized_data = [(x - min(dataBalance)) / (max(dataBalance) - min(dataBalance)) for x in dataBalance]

fig = go.Figure()
fig.add_trace(go.Box(y=normalized_data, 
                     name='Sample Data', 
                     boxpoints='all',
                     notched=True,
                     marker=dict(size=5,
                                 color='blue',)
                     ))

# Add title and labels
fig.update_layout(
    width=400,
    height=600,
    title='Box Plot Example',
    yaxis_title='Values',
)

# Display the plot
fig.show()
fig.write_image('./SampleDestribution_1.svg',  scale=2, format='svg')

[0.479, 0.31, 0.31, 0.41, 0.457, 0.405, 0.476, 0.476, 0.405, 0.374, 0.474, 0.5, 0.494, 0.444, 0.422, 0.481, 0.491, 0.444, 0.48, 0.491, 0.5, 0.5, 0.494, 0.41, 0.312, 0.386, 0.479, 0.489, 0.495, 0.384, 0.447, 0.384, 0.403, 0.403, 0.47, 0.409, 0.409, 0.312, 0.387, 0.379, 0.375, 0.417, 0.373, 0.46, 0.437, 0.39, 0.428, 0.437, 0.374, 0.428, 0.344, 0.403, 0.344, 0.482, 0.403, 0.498, 0.498, 0.405, 0.43, 0.434, 0.44, 0.458, 0.435, 0.485, 0.419, 0.425, 0.442, 0.448, 0.418, 0.402, 0.448, 0.437, 0.44, 0.405, 0.482, 0.296, 0.401, 0.471, 0.402, 0.367, 0.296, 0.442, 0.448, 0.43, 0.48, 0.416, 0.499, 0.491, 0.361, 0.444, 0.44, 0.43, 0.499, 0.425, 0.342, 0.489, 0.489, 0.419, 0.392, 0.482, 0.403, 0.44, 0.449, 0.5, 0.312, 0.386, 0.479, 0.487, 0.31]


In [84]:
# !ls ../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG/DataSummary.txt

../../ResultsSelectedMin200Samples/CombinedResults100/Splitted/resultsSVG/DataSummary.txt


In [115]:

import plotly.graph_objects as go

# Sample data
data = [10, 20, 15, 30, 25, 40, 35, 45, 50, 55, 60, 65, 70]

# Create the box plot
fig = go.Figure()

fig.add_trace(go.Box(y=data, name='Sample Data', boxpoints='all'))

# Add title and labels
fig.update_layout(
    title='Box Plot Example',
    yaxis_title='Values',
)

# Display the plot
fig.show()

In [117]:
import plotly.express as px
df = px.data.tips()
fig = px.box(df, x="time", y="total_bill")
fig.show()

In [ ]:


# min_val = min(data)
# max_val = max(data)

# normalized_data = [(x - min_val) / (max_val - min_val) for x in data]

# print(normalized_data)

In [43]:

file  = open('../../ResultsSelectedMin200Samples/CombinedResults100/merged_results_100.tsv')
lines = file.readlines()

selected_data_files  = [line.split('\t')[0] for line in lines[1:]]


import glob
import pandas as pd
import plotly.graph_objects as go

algs = ['LRC', 'DTC', 'SVC', 'RFC']


features = {}

for a  in algs:
    num_of_features = []
    fs = glob.glob('../../../../Desktop/Selected_features/SelectedFeatures/'+a+'/*.tsv')
    for f in fs:

        if f.split('/')[8] in selected_data_files:
            try:
                df = pd.read_csv(f, sep='\t')
                num_of_features.append(df.shape[1])
                # print(df.shape[1])
            except:
                print("Error in the file:", f)
                pass
    print(len(num_of_features))
    features[a] = num_of_features
    
print(features)
# # Create the box plot
# fig = go.Figure()
# fig.add_trace(go.Box(y=num_of_features, name='Sample Data', boxpoints='all'))
# # Add title and labels
# fig.update_layout(
#     width=400,
#     height=600,
#     title='Box Plot Example',
#     yaxis_title='Values',
# )
# # Display the plot
# fig.show()

hd_selected_features = []
for s in  selected_data_files:
    f = open('../../MicroBiomML_NonGit/hdlib-ra-levels1000-retrain20-cv5-feature-selection/HDC_result_files/'+s.replace('.tsv', '.tsv.txt'))
    for l in f.readlines():
        if 'Selected features:' in l:
            hd_selected_features.append(int(l.split(' ')[2]))

features['HDC'] = hd_selected_features


Error in the file: ../../../../Desktop/Selected_features/SelectedFeatures/LRC/2021-03-31.WirbelJ_2018.relative_abundance.age_category.tsv
105
106
Error in the file: ../../../../Desktop/Selected_features/SelectedFeatures/SVC/2021-03-31.OhJ_2014.relative_abundance.gender.tsv
105
106
{'LRC': [74, 615, 376, 336, 73, 196, 464, 3, 552, 284, 510, 642, 512, 356, 795, 678, 1103, 144, 94, 524, 608, 573, 547, 209, 554, 572, 38, 105, 346, 505, 83, 97, 108, 123, 545, 70, 301, 536, 63, 60, 42, 531, 166, 164, 569, 220, 100, 709, 36, 255, 119, 36, 15, 116, 287, 94, 179, 308, 88, 179, 165, 12, 231, 163, 139, 322, 288, 79, 461, 383, 152, 107, 577, 41, 32, 140, 415, 236, 306, 527, 51, 233, 339, 821, 80, 260, 563, 40, 341, 1072, 390, 194, 190, 362, 340, 51, 194, 673, 663, 54, 555, 136, 700, 105, 68], 'DTC': [312, 681, 604, 538, 543, 605, 475, 1, 617, 364, 539, 729, 535, 618, 605, 829, 713, 1111, 451, 650, 647, 644, 642, 612, 222, 648, 680, 662, 600, 504, 584, 570, 600, 574, 443, 644, 651, 369, 600, 310, 5

In [46]:
import plotly.graph_objects as go

fig = go.Figure()
# Add four box plots
fig.add_trace(go.Box(y=features['LRC'], name='LRC', boxpoints='all', width=0.5))
fig.add_trace(go.Box(y=features['DTC'], name='DTC', boxpoints='all', width=0.5))
fig.add_trace(go.Box(y=features['SVC'], name='SVC', boxpoints='all', width=0.5))
fig.add_trace(go.Box(y=features['RFC'], name='RFC', boxpoints='all', width=0.5))
fig.add_trace(go.Box(y=features['HDC'], name='HDC', boxpoints='all', width=0.5))

# Add title and labels
fig.update_layout(
    width=600,  # Adjusted width to accommodate four plots
    height=600,
    title='Selected Feature',
    yaxis_title='Number of features',
    xaxis_title='Algorithms',
    boxmode='group'  # Group the box plots side by side
)

# Display the plot
fig.write_image('../../../../Desktop/selected_Features.svg',  scale=2, format='svg')
fig.show()


In [28]:
a = '../../../../Desktop/Selected_features/SelectedFeatures/LRC/2021-03-31.WirbelJ_2018.relative_abundance.age_category.tsv'


'2021-03-31.WirbelJ_2018.relative_abundance.age_category.tsv'

17

302

241

183

158

278

103

730

355

245

137

329

112

190

222

596

86

217

354

304

29

343

348

107

125

406

428

228

11

150

237

268

43

151

60

242

180

56

242

23

114

378

182

165

168

326

337

113

193

112

30

163

116

5

79

172

154

119

463

314

333

144

95

599

101

215

21

151

151

467

167

101

29

531

135

246

203

72

16

151

2

451

169

107

820

80

193

248

75

409

320

606

253

74

15

114

451

75

221

474

192

603

19

237

107

244

